# A. To obtain quality-controlled groundwater level data from example unfiltered csv file (example_unfiltered_CGWB_data.csv) 

In [67]:
import pandas as pd
import geopandas as gpd

# Load the unfiltered CSV data of wells
unfiltered_well = pd.read_csv('/give_path_to/example_data/1_example_unfiltered_CGWB_data.csv')

# Function to check if there are at least 3 non-NaN values for a given year
def has_at_least_3_non_nan(row, year):
    columns_for_year = [f'Jan {year}', f'May {year}', f'Aug {year}', f'Nov {year}']
    non_nan_count = sum(pd.notna(row[col]) for col in columns_for_year)
    return non_nan_count >= 3

# List of years to check (2004 to 2015)
years = range(2004, 2016)

# Filter rows that meet the condition for each year (at least 3 non-NaN values)
for year in years:
    unfiltered_well = unfiltered_well[unfiltered_well.apply(lambda row: has_at_least_3_non_nan(row, year), axis=1)]

# Define the time columns to keep for each year (January, May, August, November)
time_columns = [f'{month} {year}' for year in years for month in ['Jan', 'May', 'Aug', 'Nov']]

# Filter rows based on certain conditions (no negative values, no repeating values more than twice)
def filter_rows(df):
    # Initialize a mask with all True values
    mask = pd.Series([True] * len(df), index=df.index)
    
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Extract values from columns starting from the 6th one onward (time data)
        values = row[6:]
        
        # Check if any value is negative
        if any(val < 0 for val in values):
            mask.loc[index] = False
            continue
        
        # Count occurrences of each value
        value_counts = values.value_counts()
        
        # Check if any value repeats more than twice
        if any(count > 2 for count in value_counts):
            mask.loc[index] = False
    
    # Apply the mask to filter rows
    filtered_df = df.loc[mask].reset_index(drop=True)
    return filtered_df

# Apply the filtering function
filtered_df = filter_rows(unfiltered_well)